In [1]:
# 1 - Data labeling

import numpy as np
import cv2
import os
from tqdm import tqdm

IMAGE_SIZE = (150, 150)

# 데이터 로드
def load_data(data_dir):
    dataset = 'train'
    images = []
    labels = []

    # 클래스 이름과 라벨을 얻기
    class_names_label = {name: idx for idx, name in enumerate(os.listdir(os.path.join(data_dir, dataset)))}

    print("{} 데이터 로드 중".format(dataset))

    dataset_path = os.path.join(data_dir, dataset)

    # 각 카테고리에 해당하는 폴더를 순회
    for folder in os.listdir(dataset_path):
        label = class_names_label[folder]

        # 폴더 안의 각 이미지를 순회
        folder_path = os.path.join(dataset_path, folder)
        for file in tqdm(os.listdir(folder_path)):
            # 이미지의 경로 이름을 얻기
            img_path = os.path.join(folder_path, file)

            # 이미지 열기 및 리사이즈
            image = cv2.imread(img_path)
            if image is not None:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE)

                # 이미지와 해당 라벨을 출력 리스트에 추가
                images.append(image)
                labels.append(label)

    images = np.array(images, dtype='float32')
    labels = np.array(labels, dtype='int32')

    return images, labels, class_names_label

# 데이터셋 경로 설정
data_dir = "C:/Coding/Python/machine_learning/term_project"

# 데이터 로드
images, labels, class_names_label = load_data(data_dir)

# 확인 출력
print(f'Images data : {images.shape}, dtype = {images.dtype}')
print(f'Images category : {labels.shape}, dtype = {labels.dtype}')


train 데이터 로드 중


100%|██████████| 40/40 [00:00<00:00, 2503.91it/s]

Images data : (200, 150, 150, 3), dtype = float32


Images category : (200,), dtype = int32


In [2]:
# 2 - Data Preprocessing

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# 데이터 정규화 (0-1 범위)
def normalize(data):
    return data / 255.0

# 평균 필터
def average_filter(data):
    padded_data = np.pad(data, ((1, 1), (1, 1), (0, 0)), mode='constant', constant_values=0)
    filtered_data = np.zeros_like(data)
    
    for i in range(1, padded_data.shape[0] - 1):
        for j in range(1, padded_data.shape[1] - 1):
            for k in range(padded_data.shape[2]):
                region = padded_data[i-1:i+2, j-1:j+2, k]
                filtered_data[i-1, j-1, k] = np.mean(region)
    
    return filtered_data

# 데이터 증강
def augment_data(images, labels):
    augmented_images = []
    augmented_labels = []
    
    for img, label in zip(images, labels):
        # Original
        augmented_images.append(img)
        augmented_labels.append(label)
        
        # Horizontal flip
        h_flip = cv2.flip(img, 1)
        augmented_images.append(h_flip)
        augmented_labels.append(label)
        
        # Vertical flip
        v_flip = cv2.flip(img, 0)
        augmented_images.append(v_flip)
        augmented_labels.append(label)
    
    return np.array(augmented_images), np.array(augmented_labels)

# 데이터 셔플링
def shuffle_data(images, labels):
    indices = np.arange(images.shape[0])
    np.random.shuffle(indices)
    return images[indices], labels[indices]

# 전처리 결과 확인
def display_examples(class_names, images, labels):
    fig = plt.figure(figsize=(10, 10))
    fig.suptitle("Some examples of images of the dataset", fontsize=16)

    for i in range(25):
        plt.subplot(5, 5, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
        plt.xlabel(class_names[labels[i]])
    plt.show()

# 데이터 정규화
images_normalized = normalize(images)

# 평균 필터 적용
images_filtered = np.array([average_filter(img) for img in images_normalized])

# 데이터 증강 (3배)
images_augmented, labels_augmented = augment_data(images_filtered, labels)

# 데이터 셔플링
images_shuffled, labels_shuffled = shuffle_data(images_augmented, labels_augmented)

# 전처리된 데이터 형태 출력
print(f'Images size : {images_shuffled.shape}, dtype = {labels_shuffled.dtype}')

# 학습 셋과 테스트 셋으로 분리
X_train, X_test, y_train, y_test = train_test_split(images_shuffled, labels_shuffled, test_size=0.2, random_state=42)

print("=========================================================")

# 전처리된 훈련 및 테스트 데이터 형태 출력
print(f'Train Images size : {X_train.shape}, dtype = {X_train.dtype}')
print(f'Test Images size : {X_test.shape}, dtype = {X_test.dtype}')

print("=========================================================")

print(f'Train Labels size : {y_train.shape}, dtype = {y_train.dtype}')
print(f'Test Labels size : {y_test.shape}, dtype = {y_test.dtype}')

# 전처리된 데이터 일부 이미지 출력
class_names = {idx: name for name, idx in class_names_label.items()}
display_examples(class_names, images_shuffled, labels_shuffled)

KeyboardInterrupt: 

In [ ]:
import numpy as np
import tensorflow as tf

# CNN 모델 구성
def create_cnn_model(input_shape, dropout_rates):
    model = tf.keras.Sequential([
        # First convolutional layer
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(dropout_rates[0]),
        
        # Second convolutional layer
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(dropout_rates[1]),
        
        # Flatten and fully connected layer
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(dropout_rates[2]),
        
        # Output layer
        tf.keras.layers.Dense(5, activation='softmax')
    ])
    return model

# 하이퍼파라미터 설정
dropout_rates_list = [
    [0.1, 0.1, 0.3],
    [0.2, 0.2, 0.2],
    [0.2, 0.2, 0.4],
    [0.2, 0.2, 0.5],
    [0.3, 0.3, 0.5],
    [0.4, 0.4, 0.5],
    [0.5, 0.5, 0.5],
]

batch_sizes = [16, 32, 64, 128]  # 배치 사이즈 후보

input_shape = (150, 150, 3)  # 이미지 크기와 채널 수 (RGB)

# 결과를 저장할 리스트 초기화
results = []

for dropout_rates in dropout_rates_list:
    for batch_size in batch_sizes:
        model = create_cnn_model(input_shape, dropout_rates)
        
        # 모델 컴파일
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # 모델 학습
        history = model.fit(X_train, y_train, epochs=30, batch_size=batch_size, validation_split=0.2, verbose=0)
        
        # 모델 평가
        train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
        test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
        
        # 결과 저장
        results.append({
            'dropout_rates': dropout_rates,
            'batch_size': batch_size,
            'train_acc': train_acc,
            'test_acc': test_acc
        })

# 최적의 드롭아웃 비율과 배치 사이즈 찾기
best_result = max(results, key=lambda x: x['test_acc'])

# 결과 출력
for result in results:
    print(f"Dropout Rates: {result['dropout_rates']}, Batch Size: {result['batch_size']}, Train Accuracy: {result['train_acc']:.4f}, Test Accuracy: {result['test_acc']:.4f}")

print("\nBest Dropout Rates:", best_result['dropout_rates'])
print("Best Batch Size:", best_result['batch_size'])
print(f"Best Train Accuracy: {best_result['train_acc']:.4f}, Best Test Accuracy: {best_result['test_acc']:.4f}")

# 최적의 드롭아웃 비율과 배치 사이즈로 다시 모델 학습 및 저장
best_model = create_cnn_model(input_shape, best_result['dropout_rates'])
best_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

best_model.fit(X_train, y_train, epochs=30, batch_size=best_result['batch_size'], validation_split=0.2)

# 모델 저장 경로 지정
save_path = 'C:/Coding/Python/machine_learning/term_project/models/2020144030_JMW.h5'

best_model.save('2020144030_JMW_best.h5')
print("모델이 '2020144030_JMW_best.h5' 파일로 저장되었습니다.")